In [1]:
"""Главный файл, в котором вызываются результаты всех остальных и создаётся база данных для их записи"""

import sqlite3

rdb = sqlite3.connect('Database/Result.db')
cursor = rdb.cursor()

""" Вывод результатов работы всех модулей по времени и нагрузке. """
print(f'Время выполнения (в секундах) и нагрузка (в процентах) '
      f'каждой задачи с использованием потоков:')
cursor.execute('SELECT task, time, memory FROM ThreadsResults GROUP BY task')
t_results = cursor.fetchall()
for result in t_results:
    print(f'Задача номер: {result[0]}, время: {result[1]}, нагрузка: {result[2]}')
print('')

print(f'Время выполнения (в секундах) и нагрузка (в процентах) '
      f'каждой задачи с использованием процессов:')
cursor.execute('SELECT task, time, memory FROM MultiResults GROUP BY task')
m_results = cursor.fetchall()
for result in m_results:
    print(f'Задача номер: {result[0]}, время: {result[1]}, нагрузка: {result[2]}')
print('')

print(f'Время выполнения (в секундах) и нагрузка (в процентах) '
      f'каждой задачи с использованием асинхронности:')
cursor.execute('SELECT task, time, memory FROM asyncResults GROUP BY task')
a_results = cursor.fetchall()
for result in a_results:
    print(f'Задача номер: {result[0]}, время: {result[1]}, нагрузка: {result[2]}')

Время выполнения (в секундах) и нагрузка (в процентах) каждой задачи с использованием потоков:
Задача номер: 1, время: 1.973039, нагрузка: 0.0703%
Задача номер: 2, время: 7.533068, нагрузка: 11.825803%
Задача номер: 3, время: 6.002116, нагрузка: 0.076947%
Задача номер: 4, время: 8.41999, нагрузка: 0.321407%
Задача номер: 5, время: 505.906264, нагрузка: 0.090853%

Время выполнения (в секундах) и нагрузка (в процентах) каждой задачи с использованием процессов:
Задача номер: 1, время: 0.588001, нагрузка: 0.076131%
Задача номер: 2, время: 22.435028, нагрузка: 11.836949%
Задача номер: 3, время: 6.097704, нагрузка: 0.091225%
Задача номер: 4, время: 4.822562, нагрузка: 0.334917%
Задача номер: 5, время: 90.784, нагрузка: 0.101016%

Время выполнения (в секундах) и нагрузка (в процентах) каждой задачи с использованием асинхронности:
Задача номер: 1, время: 1.955002, нагрузка: 0.114935%
Задача номер: 2, время: 13.581589, нагрузка: 0.118474%
Задача номер: 3, время: 6.002367, нагрузка: 0.11851%
Зад

In [2]:
"""Задача 1"""
import threading
import time
import sqlite3
import psutil
import tasks

cpu = psutil.Process()
threads_list = list()

rdb = sqlite3.connect('Database/Result.db')
cursor = rdb.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS ThreadsResults(
task INTEGER NOT NULL PRIMARY KEY,
time FLOAT NOT NULL,
memory TEXT NOT NULL
)
''')


def task_1():
    """

    Вызывает функцию write_words из модуля tasks.
    Создаёт потоки для выполнения функции несколько раз параллельно.
    Вычисляет время работы функции и её нагрузку на память.
    Заносит результаты вычислений в базу данных.

    """
    started_at = time.time()
    n = 0
    for i in range(1000000, 500000, -100000):
        x = threading.Thread(target=tasks.write_words, args=(i, f'file{n}.txt'))
        threads_list.append(x)
        x.start()
        n += 1
    for i in range(0, len(threads_list)):
        threads_list[i].join()
    threads_list.clear()
    ended_at = time.time()
    elapsed = round(ended_at - started_at, 6)
    memories = f'{round(cpu.memory_percent(), 6)}%'
    print(f'Время работы: {elapsed} секунд')
    print(f'Нагрузка на память: {memories}%')
    cursor.execute('INSERT or REPLACE INTO ThreadsResults (task, time, memory) '
                   'VALUES (?, ?, ?)', (1, f'{elapsed}', f'{memories}'))


task_1()
rdb.commit()
rdb.close()

Завершилась запись в файл file4.txt
Завершилась запись в файл file3.txt
Завершилась запись в файл file2.txt
Завершилась запись в файл file1.txt
Завершилась запись в файл file0.txt
Время работы: 1.755654 секунд
Нагрузка на память: 0.210864%%


In [3]:
"""Задача 2"""
import threading
import time
import math
import sqlite3
from typing import List
import psutil

lock = threading.Lock()
cpu = psutil.Process()
N_JOBS = 4
LOAD = 100_000_000

rdb = sqlite3.connect('Database/Result.db')
cursor = rdb.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS ThreadsResults(
task INTEGER NOT NULL PRIMARY KEY,
time FLOAT NOT NULL,
memory TEXT NOT NULL
)
''')


def summer(arr: List[int], i: int, summ: List[int], lock: threading.Lock) -> None:
    """

    Суммирует часть квадратных корней последовательности
    из 100000000 чисел (LOAD), полученную за счёт деления
    последовательности на N_JOBS частей.

    """
    begin = int(i * LOAD / N_JOBS)
    end = int((i + 1) * LOAD / N_JOBS)
    res = 0
    for k in range(begin, end):
        res += math.sqrt(arr[k])
    with lock:
        summ[i] = res


def task_2():
    """

    Вызывает функцию summer.
    Создаёт потоки для выполнения функции N_JOBS раз параллельно.
    Вычисляет время работы функции и её нагрузку на память.
    Заносит результаты вычислений в базу данных.

    """
    started_at = time.time()
    arr = list(range(LOAD))
    summ = [0] * N_JOBS
    threads = [None] * N_JOBS
    for i in range(N_JOBS):
        threads[i] = threading.Thread(target=summer, args=(arr, i, summ, lock))
        threads[i].start()
    for i in range(N_JOBS):
        threads[i].join()
    print(f'Резуальтат: {sum(summ)}')
    ended_at = time.time()
    elapsed = round(ended_at - started_at, 6)
    memories = f'{round(cpu.memory_percent(), 6)}%'
    print(f'Время работы: {elapsed} секунд')
    print(f'Нагрузка на память: {memories}')
    cursor.execute('INSERT or REPLACE INTO ThreadsResults (task, time, memory) '
                   'VALUES (?, ?, ?)', (2, f'{elapsed}', f'{memories}'))


task_2()
rdb.commit()
rdb.close()


Резуальтат: 666666661666.5139
Время работы: 6.153496 секунд
Нагрузка на память: 11.959611%


In [6]:
"""Задача 3"""
import threading
import time
import sqlite3
import psutil

cpu = psutil.Process()
threads_list = list()

rdb = sqlite3.connect('Database/Result.db')
cursor = rdb.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS ThreadsResults(
task INTEGER NOT NULL PRIMARY KEY,
time FLOAT NOT NULL,
memory TEXT NOT NULL
)
''')


def get_conn(host, port):
    """ Имитация соединения с некой периферией. """
    class Conn:
        def put_data(self):
            print('Отправка данных...')
            time.sleep(2)
            print('Данные отправлены.')

        def get_data(self):
            print('Получение данных...')
            time.sleep(2)
            print('Данные получены.')

        def close(self):
            print('Завершение соединения...')
            time.sleep(2)
            print('Соединение завершено.')

    print('Устанавливаем соединение...')
    time.sleep(2)
    print('Соединение установлено.')
    return Conn()


class Connection:
    """ Этот конструктор будет выполнен в заголовке with. """
    def __init__(self, host, port):
        self.host = host
        self.port = port

    def __enter__(self):
        """ Этот метод будет неявно выполнен при входе в with. """
        self.conn = get_conn(self.host, self.port)
        return self.conn

    def __exit__(self, exc_type, exc, tb):
        """ Этот метод будет неявно выполнен при выходе из with. """
        self.conn.close()


def task_3():
    """

    Создаёт потоки для параллельного выполнения функций из функции get_conn,
    применяя к ним методы класса Connection.
    Вычисляет время работы функции и её нагрузку на память.
    Заносит результаты вычислений в базу данных.

    """
    started_at = time.time()
    with Connection('localhost', 9001) as conn:
        send_task = threading.Thread(target=conn.put_data, args=())
        receive_task = threading.Thread(target=conn.get_data, args=())
        send_task.start()
        receive_task.start()
        send_task.join()
        receive_task.join()
    ended_at = time.time()
    elapsed = round(ended_at - started_at, 6)
    memories = f'{round(cpu.memory_percent(), 6)}%'
    print(f'Время работы: {elapsed} секунд')
    print(f'Нагрузка на память: {memories}')
    cursor.execute('INSERT or REPLACE INTO ThreadsResults (task, time, memory) '
                   'VALUES (?, ?, ?)', (3, f'{elapsed}', f'{memories}'))


task_3()
rdb.commit()
rdb.close()


Устанавливаем соединение...
Соединение установлено.
Отправка данных...
Получение данных...
Данные отправлены.Данные получены.

Завершение соединения...
Соединение завершено.
Время работы: 6.005368 секунд
Нагрузка на память: 0.210984%


In [7]:
"""Задача 4"""
import threading
import time
import sqlite3
import psutil
import tasks

cpu = psutil.Process()
threads_list = list()

rdb = sqlite3.connect('Database/Result.db')
cursor = rdb.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS ThreadsResults(
task INTEGER NOT NULL PRIMARY KEY,
time FLOAT NOT NULL,
memory TEXT NOT NULL
)
''')


def task_4():
    """

    Создаёт список из значений из файла Numbers.txt, разделяя на подсписки
    по разрядам (единицы, десятки, сотни и т.д.).
    Создаёт функцию writer.
    Создаёт потоки для выполнения функции несколько раз параллельно.
    Вычисляет время работы функции и её нагрузку на память.
    Заносит результаты вычислений в базу данных.

    """
    started_at = time.time()
    text_list = list()
    small_list = list()
    x = '0\n'
    n = 0
    with open('Files/Numbers.txt', encoding='utf-8') as file:
        for line in file.readlines():
            if len(line) > len(x):
                text_list.append(small_list)
                n += 1
                x = line
                small_list = []
            small_list.append(int(line))
        text_list.append(small_list)
        n += 1

    def writer(name):
        """

        Создаёт или открывает файл, в который ведёт запись чисел,
        преобразованных в текстовый формат (1 - One) через вызов
        функции number_to_words из модуля tasks.

        """
        result = open(f'Files/Text_Numbers{name}.txt', 'w+', encoding='utf-8')
        for j in range(0, len(text_list[name])):
            result.write(f'{tasks.number_to_words(text_list[name][j])}\n')
        print(f'Запись файла {name} завершена')
        result.close()

    for num in range(0, n):
        t = threading.Thread(target=writer, args=(num,))
        threads_list.append(t)
        t.start()
    for thread in threads_list:
        thread.join()
    threads_list.clear()
    ended_at = time.time()
    elapsed = round(ended_at - started_at, 6)
    memories = f'{round(cpu.memory_percent(), 6)}%'
    print(f'Время работы: {elapsed} секунд')
    print(f'Нагрузка на память: {memories}')
    cursor.execute('INSERT or REPLACE INTO ThreadsResults (task, time, memory) '
                   'VALUES (?, ?, ?)', (4, f'{elapsed}', f'{memories}'))


task_4()
rdb.commit()
rdb.close()

Запись файла 0 завершенаЗапись файла 1 завершена
Запись файла 2 завершена

Запись файла 3 завершена
Запись файла 4 завершена
Запись файла 5 завершена
Запись файла 6 завершена
Время работы: 8.23171 секунд
Нагрузка на память: 0.455263%


In [8]:
"""Задача 5"""
import threading
import time
import sqlite3
import psutil
import tasks

cpu = psutil.Process()
threads_list = list()

rdb = sqlite3.connect('Database/Result.db')
cursor = rdb.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS ThreadsResults(
task INTEGER NOT NULL PRIMARY KEY,
time FLOAT NOT NULL,
memory TEXT NOT NULL
)
''')


# Генерируем и отображаем дни рождения:
def gen_res(num_bdays):
    """

    Принимает количество дней рождений для списка.
    Генерирует список дней рождения в заданном количестве.
    С помощью функции get_match из модуля tasks проверяет список на совпавшие дни.
    После одной демонстративной проверки запускает 1000000 проверок,
    создавая случайные списки и проверяя вероятность совпадения дней рождений
    у группы людей заданного размера.
    Выводит сколько раз совпали дни рождения из 1000000 проверок и вероятность.

    """
    print(f'В списке {num_bdays} дней рождений:')
    birthdays = tasks.get_birthdays(num_bdays)
    match = tasks.get_match(birthdays)
    if match is not None:
        month_name = tasks.months[match.month - 1]
        date_text = f'{match.day} {month_name}'
        print(f'В этой симуляции несколько человек родились {date_text}')
    else:
        print(f'В этой симуляции нет человек с совпадающим днём рождения.')
    print(f'Генерация {num_bdays} случайных дней рождений 1,000,000 раз...')
    sim_match = 0
    for i in range(1_000_000):
        if tasks.get_match(tasks.get_birthdays(num_bdays)) is not None:
            sim_match = sim_match + 1
    probability = round(sim_match / 1_000_000 * 100, 2)
    print(f'Из 1,000,000 симуляций в группе из {num_bdays} людей совпадают '
          f'дни рождения {sim_match} раз. Таким образом, {num_bdays} с шансом '
          f'{probability}% будут иметь совпадающие дни рождения.')


def task_5():
    """

    Вызывает функцию gen_res.
    Создаёт потоки для выполнения функции несколько раз параллельно.
    Вычисляет время работы функции и её нагрузку на память.
    Заносит результаты вычислений в базу данных.

    """
    started_at = time.time()
    num_bdays = list()
    for i in range(2, 101, 14):
        num_bdays.append(i)
    for i in range(0, len(num_bdays)):
        t = threading.Thread(target=gen_res, args=(num_bdays[i],))
        threads_list.append(t)
        t.start()
    for thread in threads_list:
        thread.join()
    threads_list.clear()
    ended_at = time.time()
    elapsed = round(ended_at - started_at, 6)
    memories = f'{round(cpu.memory_percent(), 6)}%'
    print(f'Время работы: {elapsed} секунд')
    print(f'Нагрузка на память: {memories}')
    cursor.execute('INSERT or REPLACE INTO ThreadsResults (task, time, memory) '
                   'VALUES (?, ?, ?)', (5, f'{elapsed}', f'{memories}'))


task_5()
rdb.commit()
rdb.close()

В списке 2 дней рождений:
В этой симуляции нет человек с совпадающим днём рождения.
Генерация 2 случайных дней рождений 1,000,000 раз...
В списке 16 дней рождений:
В этой симуляции несколько человек родились 14 Июля
Генерация 16 случайных дней рождений 1,000,000 раз...
В списке 30 дней рождений:
В этой симуляции нет человек с совпадающим днём рождения.
Генерация 30 случайных дней рождений 1,000,000 раз...
В списке 44 дней рождений:
В этой симуляции несколько человек родились 14 Января
Генерация 44 случайных дней рождений 1,000,000 раз...
В списке 58 дней рождений:
В этой симуляции несколько человек родились 26 Марта
Генерация 58 случайных дней рождений 1,000,000 раз...
В списке 72 дней рождений:
В этой симуляции несколько человек родились 6 Декабря
Генерация 72 случайных дней рождений 1,000,000 раз...
В списке 86 дней рождений:
В этой симуляции несколько человек родились 22 Октября
Генерация 86 случайных дней рождений 1,000,000 раз...
В списке 100 дней рождений:
В этой симуляции нескол